<a href="https://colab.research.google.com/github/neerajthandayan/Misc./blob/main/CNNGridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN + GridSearch For Optimal Performance
#### Data Import and Pre-processing

In [ ]:
# Importing Libraries

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop

In [ ]:
#Fetching Data

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Changing the Data Shape

X_train, X_test = X_train.astype('float32') /255, X_test.astype('float32') /255
X_train, X_test = np.expand_dims(X_train, -1), np.expand_dims(X_test,-1)
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

#### Function For ConvNet

In [ ]:
# Building funtion for model

def CNN_model(act_funct='relu',opt=RMSprop,learning_rate=0.001,momentum=0.0, dropout_rate=0.0):
  #Input
  i = Input(shape=(X_train[0].shape))

  #Convolution layers
  o = Conv2D(32, (3,3), activation=act_funct)(i)
  o = MaxPooling2D((2,2))(o)
  o = Conv2D(64, (3,3), activation=act_funct)(o)
  o = MaxPooling2D((2,2))(o)
  o = Conv2D(64, (3,3), activation=act_funct)(o)

  #Dense Layer
  o = Flatten()(o)
  o = Dropout(dropout_rate)(o)
  o = Dense(64, activation=act_funct)(o)
  o = Dense(10, activation='softmax')(o)

  clf = Model(i,o)

  #Compiling Model
  clf.compile(optimizer=opt(learning_rate,momentum),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return clf

#### Checking Model Default Performance

In [ ]:
model = CNN_model()
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2) 

Epoch 1/5
750/750 [==============================] - 44s 58ms/step - loss: 0.5764 - accuracy: 0.8179 - val_loss: 0.1136 - val_accuracy: 0.9643
Epoch 2/5
750/750 [==============================] - 43s 57ms/step - loss: 0.0894 - accuracy: 0.9744 - val_loss: 0.0662 - val_accuracy: 0.9815
Epoch 3/5
750/750 [==============================] - 43s 57ms/step - loss: 0.0812 - accuracy: 0.9801 - val_loss: 0.1016 - val_accuracy: 0.9769
Epoch 4/5
750/750 [==============================] - 43s 57ms/step - loss: 0.1026 - accuracy: 0.9778 - val_loss: 0.1030 - val_accuracy: 0.9802
Epoch 5/5
750/750 [==============================] - 42s 56ms/step - loss: 0.1113 - accuracy: 0.9772 - val_loss: 0.1616 - val_accuracy: 0.9775


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Accuracy: {round(test_acc*100,2)}%')

313/313 [==============================] - 3s 10ms/step - loss: 0.1335 - accuracy: 0.9804
Accuracy: 98.04%


#### Setting Up and Fitting GridSearch

In [ ]:
act_funct = ['elu','selu','relu',tf.keras.layers.LeakyReLU()]
opt = [Adam, Adagrad, RMSprop]
learning_rate = [0.0001,0.001, 0.01, 0.1]
momentum = [0.0,0.5,0.9,0.99]
dropout_rate = [0.0,0.25,0.5]

param_grid = dict(act_funct=act_funct,opt=opt,learning_rate=learning_rate,momentum=momentum,dropout_rate=dropout_rate)
Classifier = KerasClassifier(build_fn=CNN_model,epochs=5, batch_size=64, validation_split=0.2)

grid = GridSearchCV(estimator=Classifier,param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train,y_train)

In [ ]:
#summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Checking performance with optimal parameters

optmodel = CNN_model(**grid_result.best_params_)
optmodel.fit(X_train,y_train, epochs=5, batch_size=64, validation_split=0.2)

In [ ]:
t_loss, t_acc,= optmodel.evaluate(X_test, y_test)
print(f'Accuracy: {round(t_acc*100,2)}%')